# Imports

In [1]:
import torch
!pip install torch-geometric torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 28.0 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=6da50ce46b432be5d90d644275da54d6b6c7978eaa605f7c37e6dd088d232fe7
  Stored in directory: /root/.cache/pip/wheels/31/b2/8c/9b4bb72a4384eabd1ffeab2b7ead692c9165e35711f8a9dc72
Successfully built torch-geometric
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [2]:
import multiprocessing as mp
import os
import pickle
import warnings
import torch
from functools import partial
from itertools import combinations
import numpy as np
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler, normalize
from tqdm import tqdm

from scipy.linalg import orthogonal_procrustes
from torch_geometric.datasets import Planetoid

In [3]:
# use google drive for saving and loading information
from google.colab import drive
import pickle
import os

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/L45_project/'

Mounted at /content/drive


# Cora Data Set Stats

In [4]:
cora_dataset = Planetoid("/tmp/cora", name="cora", split="full")
cora_data = cora_dataset[0]

Processing...
Done!


In [5]:
# Create a random list of indices with 10 items from each class for 2nd cossim
labels = cora_dataset[0].y.detach().numpy()
num_classes = 7
num_samples = 10
cora_indices = []
for i in range(num_classes):
  class_i = np.random.choice(np.where(labels == i)[0], size=num_samples, replace=False)
  print(class_i)
  cora_indices += class_i.tolist()


[2267 2052  863 2181 1841 2098 1696 1172   10  437]
[2324  303 1797 2079 1251  834 2096  138 2093 1638]
[1719  227  978  800 2597 2448  526 2585 2612 1433]
[2207 2343 2033  938 1502 1106 2693  681 1100   25]
[2355 1102 2399 2294  375  810 2671  492  585  582]
[1840  454  119  517 1314  728  117  472 1455 2017]
[1420 1170 2158 1215 1387 1085 1359  847 1504  499]


# Metric Calculations

In [6]:
# Adapted from https://github.com/SGDE2020/embedding_stability/blob/master/lib/tools/comparison.py

# This potentially solves the issue of the multithreading never returning https://github.com/ipython/ipython/issues/12396
# mp.set_start_method('fork')

class Comparison:

    def __init__(self, emb_dir, embeddings, num_nodes, file_prefix):
        """
        emb_dir: str, path where embeddings are saved
        embeddings: list, list of strings that specify the embedding files
        """
        self.dir = emb_dir
        self.embeddings = embeddings
        self.pairs = self._combinations(embeddings)
        self.num_vertices = num_nodes

        # use file names without numbering to mark result files
        self.file_prefix = file_prefix

    def _combinations(self, emb_list):
        """ Computes all different comparison pairs of a list of embeddings """
        return [pair for pair in combinations(emb_list, 2)]

    # KEEP
    def _analyse_jaccard(self, queries, nodes, k, pair):
        """This function is called in the multiprocessing of jaccard score"""
        indices_0 = np.asarray(queries[pair[0]])
        indices_1 = np.asarray(queries[pair[1]])
        nodes = np.asarray(nodes)
        jaccard_score = {}
        for i in range(len(nodes)):
            jaccard_score[nodes[i]] = \
                len(np.intersect1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)], assume_unique=True)) / \
                len(np.union1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)]))
        return list(jaccard_score.values())

    # KEEP
    def _analyse_second_cossim(self, queries, normed_embs, nodes, k, pair):
        """
        This function is called in the multiprocessing of the second order cosine similarity.
        """

        # Convert the indices of nearest neighbors back into numpy
        indices_0 = np.asarray(queries[pair[0]])
        indices_1 = np.asarray(queries[pair[1]])

        # Convert the embeddings and nodes back into numpy
        norm_emb_0 = np.asarray(normed_embs[pair[0]])
        norm_emb_1 = np.asarray(normed_embs[pair[1]])
        nodes = np.asarray(nodes)

        # Compute the second order cosine similarity
        pair_results = []
        for i in range(len(nodes)):
            # Build the set of nearest neighbors w.r.t. both embeddings
            # Use indices from 1 to k+1, because the first entry will always be the node itself
            neighbors_union = np.union1d(indices_0[i, 1:(k + 1)], indices_1[i, 1:(k + 1)])

            # Vectors of cosine similarity values of nearest neighbors. There was an error in the original source code, did not use norm_emb_1 for m1
            m0 = cos_sim(norm_emb_0[neighbors_union], norm_emb_0[nodes[i]].reshape(1, -1))
            m1 = cos_sim(norm_emb_1[neighbors_union], norm_emb_1[nodes[i]].reshape(1, -1))

            assert m0.shape[1] == 1 and m1.shape[1] == 1, "m0 and m1 should only have a single variable in the second dimension"
            m0 = m0.flatten()
            m1 = m1.flatten()

            # Cosine similarity between similarity vectors
            # pair_results.append(float(np.dot(m0, m1) / (np.linalg.norm(m0) * np.linalg.norm(m1))))
            pair_results.append(float(1-cosine(m0,m1)))
        return pair_results

    # KEEP
    def k_nearest_neighbors(self, nodes=None, append=False, samples=100, k=10, jaccard=False, load=False,
                            kload_size=100, save=True, save_path=None, num_processes=4):
        """
        Computes the k nearest neighbors to some specified nodes with respect to cosine similarity. As an intermediate
        step, it computes the 100 nearest neighbors and saves them to file. Based on these neighbors, the k-nn overlaps
        are computed.
        Args:
            nodes: list, that specifies the nodes
            append: bool, if nodes are specified, whether additional nodes will be sampled and added to nodes
            samples: int, number of nodes that will be sampled
            k: int, number of neighbors that will be used in the comparison
            jaccard: bool, whether jaccard score or overlap will be used as similarity measure
            load: bool, whether a file of computed neighbors will be used
            kload_size: Size of k in knn file to load
            save: bool, whether the results should be saved in a text file
            save_path: str, path where the file will be saved
            num_processes: number of random processes in parallelization
        Returns:
            dict, "nodes": array of nodes, "overlaps": array of overlaps, columns are values per node; or array of
            jaccard scores
        """
        # Handle the nodes input: Whether to sample nodes, use given nodes, or both.
        nodes = self._get_nodes(nodes, samples, append)

        # Load k nearest neighbors to save time? Else we will have to compute them first.
        if load:
            file_name = self.file_prefix + "_" + str(kload_size) + "nns.pickle"
            with open(os.path.join(save_path, file_name), "rb") as pickle_file:
                queries = pickle.load(pickle_file)
            assert list(queries.keys()) == self.embeddings, ("Keys of loaded queries do not match"
                                                             " with available embeddings")

        # Normalizing the embeddings to be able to use distance as a proxy for cosine similarity
        # BallTree from sklearn is used to compute the neighbors efficiently
        else:
            queries = self.nearest_neighbors_queries(nodes, k, save_path)
        # Store the naive overlaps for all pairs

        print(f"\n\n {queries.keys()} \n")

        # Use multiprocessing to speed up overlap computation.
        # Too much data is passed to the processes which makes it inefficient.
        # Possibly, it is faster to store the data as a file as an intermediate step.
        # only run if less than 100 neighbors are queried, as too large neighborhoods may cause memory issues when
        # distributing tasks
        if num_processes > 1 and k <= 100:
            with mp.Pool(num_processes) as p:
                # arguments passed in multiprocessing must be pickable
                p_queries = queries
                p_nodes = nodes.tolist()
                if jaccard:
                    multiprocess_func = partial(self._analyse_jaccard, p_queries, p_nodes, k)
                else:
                    multiprocess_func = partial(self._analyse_knn, p_queries, p_nodes, k)
                # li_overlap = p.map(multiprocess_func, self.pairs)
                li_overlap = []
                for result in tqdm(p.imap(multiprocess_func, self.pairs), total=len(self.pairs)):
                  li_overlap.append(result)
        else:
            if jaccard:
                li_overlap = [self._analyse_jaccard(queries, nodes.tolist(), k, pair) for pair in self.pairs]
            else:
                li_overlap = [self._analyse_knn(queries, nodes.tolist(), k, pair) for pair in self.pairs]

        # Convert the result into numpy
        overlap = np.asarray(li_overlap)

        # Save the results
        if jaccard:
            nodes_suffix = "jaccard_nodes"
            scores_suffix = f"{k}nn_jaccard"
        else:
            nodes_suffix = "overlap_nodes"
            scores_suffix = f"{k}nn_overlap"
        if save is True:
            np.save(os.path.join(save_path, f"{self.file_prefix}_{nodes_suffix}"), nodes)
            np.save(os.path.join(save_path, f"{self.file_prefix}_{scores_suffix}"), overlap)
        return {"nodes": nodes, "overlaps": overlap}

    def jaccard_similarity(self, nodes=None, append=False, samples=100, k=10, load=False, kload_size=100, save=True,
                           save_path=None, num_processes=4):
        """
        Alias for k_nearest_neighbors with jaccard=True.
        See k_nearest_neighbors for detailed documentation.
        """
        return self.k_nearest_neighbors(nodes=nodes, append=append, samples=samples, k=k, jaccard=True, load=load,
                                        kload_size=kload_size, save=save, save_path=save_path,
                                        num_processes=num_processes)


    def second_order_cosine_similarity(self, nodes=None, append=False, num_samples=1000, k=10, load=True, save=False,
                                       save_path=None, num_processes=4):
        """ Computes second order cosine similarity.
        Args:
            nodes: list, that specifies the nodes
            append: bool, if nodes are specified, whether additional nodes will be sampled and added to nodes
            num_samples: int, number of nodes that will be sampled
            k: int, number of neighbors that will be used in the comparison
            load: bool, whether to load nearest neighbors from file
            save: bool, whether the results should be saved
            save_path: str, path where the file will be saved
            num_processes: number of random processes in parallelization
        Returns:
            nodes: numpy array of used nodes
            results: numpy array of similarity values of size (number of embedding pairs, number of nodes)
        """

        # Handle the nodes input: Whether to sample nodes, use given nodes, or both.
        nodes = self._get_nodes(nodes, num_samples, append)

        # Load required data: nearest neighbors, embeddings
        normed_embs = {}

        if load:
            # Load nearest neighbors from file
            file_name = self.file_prefix + "_" + str(k)+ "nns.pickle"
            with open(os.path.join(save_path, file_name), "rb") as pickle_file:
                queries = pickle.load(pickle_file)
            assert list(queries.keys()) == self.embeddings, ("Keys of loaded queries do not match with "
                                                             "available embeddings")
            for emb in tqdm(self.embeddings, desc="Loading nearest neighbor files"):
                normed_embs[emb] = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2', copy=False)
        else:
            queries, normed_embs = self.nearest_neighbors_queries(nodes, k, save_path, return_embeddings=True)

        # Start computation of second order cosine similarity
        # arguments passed in multiprocessing must be pickable
        p_normed_embs = dict([(key, norm_emb.tolist()) for key, norm_emb in normed_embs.items()])
        p_nodes = nodes.tolist()
        if num_processes > 1 and k <= 100:
            with mp.Pool(num_processes) as p:
                li_results = p.map(partial(self._analyse_second_cossim, queries, p_normed_embs, p_nodes, k), self.pairs)
              # li_results = []
              # partial_func = partial(self._analyse_second_cossim, queries, p_normed_embs, p_nodes, k) 
              # for result in tqdm(p.imap(partial_func, self.pairs), total=len(self.pairs)):
              #   li_results.append(result)
        else:
          li_results = []
          for pair in tqdm(self.pairs, desc="Comparing embeddings"):
            li_results.append(self._analyse_second_cossim(queries, p_normed_embs, p_nodes, k, pair))

        results = np.asarray(li_results)

        if save is True:
            np.save(os.path.join(save_path, f"{self.file_prefix}_{k}nn_2nd_order_cossim"), results)

        return nodes, results

    # KEEP 
    def nearest_neighbors_queries(self, nodes, k, save_path, return_embeddings=False):
        """Uses a ball tree to compute the nearest neighbors in the embedding space"""
        queries = {}
        normed_embs = {}
        # Normalizing the embeddings to be able to use distance as a proxy for cosine similarity
        # BallTree from sklearn is used to compute the neighbors efficiently
        if return_embeddings:
            for emb in tqdm(self.embeddings, desc="Querying nearest neighbors"):
                normed_embs[emb] = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2', copy=False)
                ball_tree = BallTree(normed_embs[emb], leaf_size=40)
                queries[emb] = ball_tree.query(normed_embs[emb][nodes, :], k=k + 1, return_distance=False).tolist()
        else:
            for emb in tqdm(self.embeddings, desc="Querying nearest neighbors"):
                normalized_embedding = normalize(self.read_embedding(os.path.join(self.dir, emb)), norm='l2',
                                                 copy=False)
                ball_tree = BallTree(normalized_embedding, leaf_size=40)
                # Query the k+1 nearest neighbors, because a node will always be the closest neighbor to itself
                queries[emb] = ball_tree.query(normalized_embedding[nodes, :], k=k + 1, return_distance=False).tolist()

        # Save the computed neighbors to be able to skip the computation
        self.save_pickle(queries, save_path, self.file_prefix + "_" + str(k) + "nns")

        if return_embeddings:
            return queries, normed_embs
        else:
            return queries

    # Don't need this
    def sample_nodes(self, k):
        """
        Sample unique nodes of an embedding
        Args:
            k: int, number of nodes to sample
        Returns:
            numpy array of node ids of length k if k is smaller than the number of nodes available.
            Otherwise, all available nodes are returned.
        """
        vertices = np.arange(self.num_vertices)
        np.random.shuffle(vertices)
        return vertices[:min(k, self.num_vertices)]


    def cossim_analysis(self, nodes=None, save_path=None, center=False):
        """ Computes aligned cosine similarity values. Internally performs orthogonal transformation (Procrustes
        problem) between two embeddings and saves transformation matrices as well as vector of resulting errors
        """

        # Warn about deprecated arguments
        if nodes is not None:
            warnings.warn("nodes argument is deprecated. All nodes are used instead.", DeprecationWarning)

        # Set up file naming
        if center is True:
            # matrix_name = "linQMatrix"
            results_suffix = "linproc_cossim"
        else:
            # matrix_name = "QMatrix"
            results_suffix = "aligned_cossim"

        # Read the embeddings
        normed_embs = {}
        for emb in tqdm(self.embeddings, desc="Reading embeddings"):
            normed_embs[emb] = normalize(
                    self.read_embedding(os.path.join(self.dir, emb))[np.arange(self.num_vertices)], norm='l2',
                    copy=False)

        # Do the analysis
        emb_ind = -1
        results = []
        for pair in tqdm(self.pairs, desc="Comparing embeddings"):

            # only update first embedding if it does not change
            if emb_ind != pair[0]:
                emb_ind = pair[0]

            W1 = normed_embs[emb_ind]

            # transform W2 into W1 -> load matrix from previous step.
            # emb_number_0 = pair[0].split("_")[1]
            # emb_number_1 = pair[1].split("_")[1]
            Q, _ = orthogonal_procrustes(normed_embs[pair[1]], normed_embs[pair[0]], check_finite=False)
            W2 = normed_embs[pair[1]].dot(Q)

            # Do 1-cosine to get the actual cosine because of the method we use
            pair_results = np.array([1-cosine(W1[i], W2[i]) for i in range(self.num_vertices)])
            results.append(pair_results)

        results = np.asarray(results)
        np.save(os.path.join(save_path, f"{self.file_prefix}_{results_suffix}"), results)
        return np.arange(self.num_vertices), results

    def _get_nodes(self, nodes, num_samples, append):
        """
        Handles getting nodes for the experiments.
        Args:
            nodes: list, node ids
            num_samples: int, how many nodes should be sampled
            append: bool, whether to append sampled nodes to specified nodes
        Returns:
            numpy array of (sampled) node ids
        """
        if nodes is None:
            nodes = self.sample_nodes(num_samples)
        elif append is True:
            # allows specified nodes to be taken twice
            nodes.extend(self.sample_nodes(num_samples))
        return np.asarray(nodes)

    def save_pickle(self, obj, save_path, file_name):
        if save_path is None:
            save_path = os.getcwd()
        if file_name is None:
            # generate name of report from embedding input: use name of first embedding file without number information
            file_name = self.file_prefix
        with open(os.path.join(save_path, f"{file_name}.pickle"), "wb") as f:
            pickle.dump(obj, f)

    def get_combinations(self):
        return self.pairs

    def get_vertex_count(self):
        return self.num_vertices

    # the original code reorders the nodes by ID, but our nodes have the same ordering so there is no need to reorder
    def read_embedding(self, path):
      node_embedding = torch.load(path).detach().numpy()
      return node_embedding

In [7]:
# Code adapted from https://github.com/SGDE2020/embedding_stability/blob/master/similarity_tests/similarity_tests.py
TESTS= ["cossim", "knn", "jaccard", "ranks", "rns", "angdiv", "2ndcos", "orthtf", "cos", "lintf", "lincos"]

# Expects the the file names in the embedding directory to be of the form model-name_seed_emb.pt
def run_tests(embedding_dir, file_prefix, tests=TESTS, num_nodes=-1, knn_size=20, load_knn=False, kload_size=None,
                results_dir=None, num_processes=4, cossim_sec_indices=None):
    """
    Run specified similarity tests.
    Params:
        embedding_dir: str, path to directory where embeddings are stored
        file_prefix: the prefix for files for saving results
        tests: list of str, tests that will be conducted
        num_nodes: int, number of nodes of a graph that are used in the tests (just the number of nodes in the graph)
        knn_size: int, neighborhood size for knn test
        load_knn: bool, whether a stored knn matrix should be loaded
        nodeinfo_dir = str, path to directory where nodeinfo is stored (tables)
        results_dir = str, path to directory where results will be saved to
    """
    # Store results in the embedding directory in a results file if no other location is specified
    if results_dir is None:
      results_dir = embedding_dir + "results/"
    # Create results directory if it does not exist
    if not os.path.exists(results_dir):
        os.mkdir(results_dir) 

    # Computes a list of all _emb.pt files in a given directory. Assumes the directory being used only contains results from one architecture and data set
    fnames = sorted([f for f in os.listdir(embedding_dir) if f.endswith("_emb.pt")])
    print(fnames)

    if len(fnames) <= 1:
        print(f"Did not find any embeddings for in directory {embedding_dir}")
 
    else:
        # To be compatible with the original source code, we need a list of nodes indices we will be using in our metric computations
        # We use all the nodes
        nodes = [i for i in range(num_nodes)]

        # Allow us to use specific nodes for the second order cossine similarity as it is very computational expensive
        cossim_nodes = nodes if cossim_sec_indices is None else cossim_sec_indices

        if load_knn and kload_size is None:
            kload_size = knn_size

        # Start tests
        comp = Comparison(emb_dir=embedding_dir, embeddings=fnames, num_nodes=num_nodes, file_prefix=file_prefix)
        if "cossim" in tests:
            print("Aligned cosine similarity")
            comp.cossim_analysis(nodes=nodes, save_path=results_dir)
        if "jaccard" in tests:
            print("Executing jaccard score")
            comp.jaccard_similarity(
                nodes=nodes, append=False, k=knn_size,
                load=load_knn, kload_size=kload_size, save=True, save_path=results_dir, num_processes=num_processes
            )
        if "2ndcos" in tests:
            print("Executing second order cosine similarity")
            comp.second_order_cosine_similarity(
                nodes=cossim_nodes, append=False, k=knn_size,
                save=True, save_path=results_dir, num_processes=num_processes, load=load_knn
            )



# Run metric calculations

In [8]:
NUM_NODES_CORA = 2708
# CHANGE these to match you directory naming structure
# model_directory = "GATV2/"
# model_file_prefix = "GATV2_analysis"
model_directory = "GraphSAGE/"
model_file_prefix = "GraphSAGE_analysis"

In [9]:
# Run the tests for tests=["cossim", "jaccard"], multiple processes work for these
run_tests(embedding_dir=file_path+model_directory, file_prefix=model_file_prefix, tests=["cossim", "jaccard"], num_nodes=NUM_NODES_CORA, num_processes=4)

<ipython-input-6-86dd3d9fc16c>:271: DeprecationWarning: nodes argument is deprecated. All nodes are used instead.
  warnings.warn("nodes argument is deprecated. All nodes are used instead.", DeprecationWarning)


['GraphSAGE_1024258131_emb.pt', 'GraphSAGE_116846604_emb.pt', 'GraphSAGE_1221215631_emb.pt', 'GraphSAGE_1517964140_emb.pt', 'GraphSAGE_170173784_emb.pt', 'GraphSAGE_1818027900_emb.pt', 'GraphSAGE_1860537279_emb.pt', 'GraphSAGE_1863727779_emb.pt', 'GraphSAGE_2036056847_emb.pt', 'GraphSAGE_2105922959_emb.pt', 'GraphSAGE_2135956485_emb.pt', 'GraphSAGE_2342954646_emb.pt', 'GraphSAGE_2455059856_emb.pt', 'GraphSAGE_2739899259_emb.pt', 'GraphSAGE_2794978777_emb.pt', 'GraphSAGE_2952735006_emb.pt', 'GraphSAGE_3300171104_emb.pt', 'GraphSAGE_3303475786_emb.pt', 'GraphSAGE_361232447_emb.pt', 'GraphSAGE_3647665043_emb.pt', 'GraphSAGE_3692371949_emb.pt', 'GraphSAGE_3710910636_emb.pt', 'GraphSAGE_400225693_emb.pt', 'GraphSAGE_4083009686_emb.pt', 'GraphSAGE_4193977854_emb.pt', 'GraphSAGE_516507873_emb.pt', 'GraphSAGE_572297925_emb.pt', 'GraphSAGE_806299656_emb.pt', 'GraphSAGE_880019963_emb.pt', 'GraphSAGE_89475662_emb.pt']
Aligned cosine similarity


Comparing embeddings: 100%|██████████| 435/435 [01:30<00:00,  4.81it/s]


Executing jaccard score


Querying nearest neighbors: 100%|██████████| 30/30 [00:20<00:00,  1.43it/s]



 dict_keys(['GraphSAGE_1024258131_emb.pt', 'GraphSAGE_116846604_emb.pt', 'GraphSAGE_1221215631_emb.pt', 'GraphSAGE_1517964140_emb.pt', 'GraphSAGE_170173784_emb.pt', 'GraphSAGE_1818027900_emb.pt', 'GraphSAGE_1860537279_emb.pt', 'GraphSAGE_1863727779_emb.pt', 'GraphSAGE_2036056847_emb.pt', 'GraphSAGE_2105922959_emb.pt', 'GraphSAGE_2135956485_emb.pt', 'GraphSAGE_2342954646_emb.pt', 'GraphSAGE_2455059856_emb.pt', 'GraphSAGE_2739899259_emb.pt', 'GraphSAGE_2794978777_emb.pt', 'GraphSAGE_2952735006_emb.pt', 'GraphSAGE_3300171104_emb.pt', 'GraphSAGE_3303475786_emb.pt', 'GraphSAGE_361232447_emb.pt', 'GraphSAGE_3647665043_emb.pt', 'GraphSAGE_3692371949_emb.pt', 'GraphSAGE_3710910636_emb.pt', 'GraphSAGE_400225693_emb.pt', 'GraphSAGE_4083009686_emb.pt', 'GraphSAGE_4193977854_emb.pt', 'GraphSAGE_516507873_emb.pt', 'GraphSAGE_572297925_emb.pt', 'GraphSAGE_806299656_emb.pt', 'GraphSAGE_880019963_emb.pt', 'GraphSAGE_89475662_emb.pt']) 




100%|██████████| 435/435 [01:53<00:00,  3.83it/s]


In [10]:
# Run the tests for tests=["2ndcos"], multiple processes does not work for this so only run on 1, but we can load the nearest neighbours from the previous metric calculations
run_tests(embedding_dir=file_path+model_directory, file_prefix=model_file_prefix, tests=["2ndcos"], num_nodes=NUM_NODES_CORA, num_processes=1, load_knn=True)

['GraphSAGE_1024258131_emb.pt', 'GraphSAGE_116846604_emb.pt', 'GraphSAGE_1221215631_emb.pt', 'GraphSAGE_1517964140_emb.pt', 'GraphSAGE_170173784_emb.pt', 'GraphSAGE_1818027900_emb.pt', 'GraphSAGE_1860537279_emb.pt', 'GraphSAGE_1863727779_emb.pt', 'GraphSAGE_2036056847_emb.pt', 'GraphSAGE_2105922959_emb.pt', 'GraphSAGE_2135956485_emb.pt', 'GraphSAGE_2342954646_emb.pt', 'GraphSAGE_2455059856_emb.pt', 'GraphSAGE_2739899259_emb.pt', 'GraphSAGE_2794978777_emb.pt', 'GraphSAGE_2952735006_emb.pt', 'GraphSAGE_3300171104_emb.pt', 'GraphSAGE_3303475786_emb.pt', 'GraphSAGE_361232447_emb.pt', 'GraphSAGE_3647665043_emb.pt', 'GraphSAGE_3692371949_emb.pt', 'GraphSAGE_3710910636_emb.pt', 'GraphSAGE_400225693_emb.pt', 'GraphSAGE_4083009686_emb.pt', 'GraphSAGE_4193977854_emb.pt', 'GraphSAGE_516507873_emb.pt', 'GraphSAGE_572297925_emb.pt', 'GraphSAGE_806299656_emb.pt', 'GraphSAGE_880019963_emb.pt', 'GraphSAGE_89475662_emb.pt']
Executing second order cosine similarity


Comparing embeddings: 100%|██████████| 435/435 [11:25<00:00,  1.58s/it]


# Compute metric averages

In [11]:
def read_and_average_metric(file_suffix):
  results_arr = np.load(file_path+model_directory+"/results/"+model_file_prefix+"_"+file_suffix+".npy", mmap_mode='r')
  results_flat = results_arr.flatten() 
  # Compute the 5 number summary for the data across all classes, better than mean and std for potentially skewed distribution
  q1, q2, q3 = np.percentile(results_flat, [25,50,75])
  min, max = results_flat.min(), results_flat.max()
  # Print results
  print("TOTAL STATS")
  print("q1=" + str(q1) + ", q2=" + str(q2) + ", q3=" + str(q3) + ", min=" + str(min) + ", max=" + str(max))

  for i in range(num_classes):
    indices = np.where(labels==i)[0]
    # Filter the results by class
    class_vals = results_arr[:, indices]
    # Compute stats for the class
    q1_c, q2_c, q3_c = np.percentile(class_vals, [25,50,75])
    min_c, max_c = class_vals.min(), class_vals.max()
    # Print results
    print("CLASS STATS " + str(i))
    print("q1=" + str(q1_c), ", q2=" + str(q2_c) + ", q3=" + str(q3_c) + ", min=" + str(min_c) + ", max=" + str(max_c))


In [12]:
# Read and compute aligned cosine metric
read_and_average_metric("aligned_cossim")

TOTAL STATS
q1=0.9819027185440063, q2=0.9877340793609619, q3=0.9914834499359131, min=0.8200086951255798, max=0.9986557364463806
CLASS STATS 0
q1=0.9776468873023987 , q2=0.9841133952140808, q3=0.9882989525794983, min=0.8290441632270813, max=0.9971880316734314
CLASS STATS 1
q1=0.9791691303253174 , q2=0.9850701689720154, q3=0.9891679286956787, min=0.9052060842514038, max=0.9973332285881042
CLASS STATS 2
q1=0.9847258478403091 , q2=0.9891674518585205, q3=0.9921981692314148, min=0.8738217353820801, max=0.9986557364463806
CLASS STATS 3
q1=0.9851939976215363 , q2=0.9899925291538239, q3=0.9929153770208359, min=0.8607029318809509, max=0.9982352256774902
CLASS STATS 4
q1=0.9844665825366974 , q2=0.9895354211330414, q3=0.992341548204422, min=0.8739640712738037, max=0.997797966003418
CLASS STATS 5
q1=0.9792811572551727 , q2=0.9851621091365814, q3=0.9890294522047043, min=0.8567337989807129, max=0.996964156627655
CLASS STATS 6
q1=0.9775650948286057 , q2=0.9829839169979095, q3=0.9873583316802979, min=0

In [13]:
# Read and compute jaccard metric
read_and_average_metric("20nn_jaccard")

TOTAL STATS
q1=0.6, q2=0.6666666666666666, q3=0.7391304347826086, min=0.0, max=1.0
CLASS STATS 0
q1=0.6 , q2=0.6666666666666666, q3=0.7391304347826086, min=0.08108108108108109, max=1.0
CLASS STATS 1
q1=0.6666666666666666 , q2=0.7391304347826086, q3=0.8181818181818182, min=0.21212121212121213, max=1.0
CLASS STATS 2
q1=0.6 , q2=0.6666666666666666, q3=0.7391304347826086, min=0.05263157894736842, max=1.0
CLASS STATS 3
q1=0.5384615384615384 , q2=0.6, q3=0.6666666666666666, min=0.0, max=1.0
CLASS STATS 4
q1=0.6 , q2=0.6666666666666666, q3=0.7391304347826086, min=0.1111111111111111, max=1.0
CLASS STATS 5
q1=0.6 , q2=0.6666666666666666, q3=0.7391304347826086, min=0.08108108108108109, max=1.0
CLASS STATS 6
q1=0.6 , q2=0.6666666666666666, q3=0.7391304347826086, min=0.14285714285714285, max=1.0


In [14]:
# Read and compute 2nd order cosine similarity metric
read_and_average_metric("20nn_2nd_order_cossim")

TOTAL STATS
q1=0.9998985682566216, q2=0.9999612323654217, q3=0.999982855051121, min=0.9655953574429701, max=0.9999997226165889
CLASS STATS 0
q1=0.9997806039487913 , q2=0.9999004278558038, q3=0.9999496186267679, min=0.9879120146786048, max=0.9999983769949087
CLASS STATS 1
q1=0.9998653278153364 , q2=0.999939111506851, q3=0.9999678460617669, min=0.9789721034604345, max=0.9999978442734464
CLASS STATS 2
q1=0.9999507453975086 , q2=0.9999792810892221, q3=0.9999903868428818, min=0.9930411322326007, max=0.9999997226165889
CLASS STATS 3
q1=0.9999405641880775 , q2=0.9999745594469109, q3=0.9999876745941372, min=0.9857364882702313, max=0.9999994263006915
CLASS STATS 4
q1=0.9999361423797406 , q2=0.9999714812007284, q3=0.9999853418729661, min=0.9675720024707195, max=0.9999994168188495
CLASS STATS 5
q1=0.9998131037463389 , q2=0.9999279567556802, q3=0.9999699247809644, min=0.9831476570647086, max=0.9999977781237698
CLASS STATS 6
q1=0.9998483214861353 , q2=0.9999366388248472, q3=0.9999665364943077, min=

To do:
- Store the nodes being used if we're only using a subset
- Subsets for second_order_cosine_similarity for larger dataset